In [16]:
from general_utils.reader import JSONLineReader
from sklearn.metrics import classification_report
from dataset.def_dataset import Fact
from config import PROJECT_DIR

In [26]:
output = JSONLineReader().read(PROJECT_DIR.joinpath('dataset/openai/output_german_contextless-gpt3_5-turbo.jsonl'))

In [27]:
gt_labels = []
pr_labels = []
for entry in output:
    predicted = entry['predicted']
    pr_labels.append(Fact[entry['predicted']].to_factuality())
    gt_labels.append(Fact[entry['label']].to_factuality())

In [28]:
print(classification_report(gt_labels, pr_labels, zero_division=0, digits=3))

              precision    recall  f1-score   support

           0      0.727     0.225     0.344       355
           1      0.542     0.915     0.681       355

    accuracy                          0.570       710
   macro avg      0.634     0.570     0.512       710
weighted avg      0.634     0.570     0.512       710

